In [ ]:
# """ Mount Google Drive to Google Colab Notebook
# """ 
# from google.colab import drive
# drive.mount('/content/drive')
# """ Change present working directory
# """
# %cd /content/drive/MyDrive/CS5260/Github/

In [ ]:
# %cd CS5260
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import pandas as pd
import skimage.io as io
import matplotlib.pyplot as plt
import torch.utils.data as data
import torch
import json
import os
import nltk
import random
from tqdm import tqdm
tqdm.pandas()
nltk.download('stopwords')
nltk.download('punkt')
%matplotlib inline

In [ ]:
%load_ext autoreload
%autoreload
from src.utils.data_loader import get_loader

In [ ]:
with open('./data/vaild_dataset.json', 'r') as outfile:
  product_info = pd.read_json(json.load(outfile), orient="records")
print(product_info.shape)
product_info.head()

In [ ]:
product_info["perCategory"].unique()

In [ ]:
with open('./data/vaild_dataset.json', 'r') as outfile:
  product_info = pd.read_json(json.load(outfile), orient="records")

import re
def get_space_len(s):
  return len(max(re.findall(' +', s), key=len, default=[0]))


product_info = product_info[product_info.valid == True]
product_info = product_info[product_info["perCategory"].isin(["AMAZON_FASHION", 'All_Beauty',
                                                               'Toys_and_Games','Office_Products',
                                                              'Home_and_Kitchen','Electronics',
                                                              'Clothing_Shoes_and_Jewelry'])]
product_info["file_name"] = product_info["imageURLHighRes"].str.split("/").str[-1]

product_info = product_info.explode("description")
product_info['description'] = product_info['description'].str.replace(r'<[^<>]*>', '', regex=True)
product_info["description"] = product_info["description"].str.split(r'\;|\!|\?|\.|\-')
product_info = product_info.explode("description")
product_info["description"] = product_info["description"].str.split(r'\n')
product_info = product_info.explode("description")
product_info["description"] = product_info["description"].str.lstrip(r'\,')
# product_info["description"] = product_info["description"].str.split(r'<br>|</td>')
# product_info = product_info.explode("description")

product_info = product_info.where(
    (product_info["description"].str.len()>50) & 
    (product_info["description"].astype(str).apply(get_space_len)<4) & 
    (product_info["description"] != "") &
    (not "<td" in product_info["description"])
    ).dropna().reset_index(drop=True)
product_info["description"] = product_info["description"].str.strip()
product_info['sentence_index'] = product_info.groupby('file_name').cumcount()  # Enumerate Groups
product_info = product_info[product_info['sentence_index'] <= product_info["description"].str.len().min()].reset_index(drop=True)
print(product_info.shape)
product_info.head(5)

# for descript in product_info.loc[:10,"description"]:
#     print("="*100)
#     print(descript)

In [ ]:
idex = random.randint(0, product_info.shape[0])
img_url = product_info.loc[idex,"imageURLHighRes"]
caption = product_info.loc[idex,"description"]
Category = product_info.loc[idex,"perCategory"]
print(caption)
print(img_url)
print(Category)
I = io.imread(img_url)
plt.axis('off')
plt.imshow(I)
plt.show()

# Training Setup 

In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms
import numpy as np
import sys
import os
import math
import pickle
from sklearn.model_selection import train_test_split
import torch.utils.data as data
import nltk
from nltk.translate.bleu_score import corpus_bleu
nltk.download('punkt')

%load_ext autoreload
%autoreload
from src.utils.data_loader import get_loader
from src.models.CNN_Encoder import EncoderCNN
from src.models.RNN_decoder_catAttention import DecoderRNN
from src.models.MLP_Encoder import MlpEncoder
from src.models.trainer import train
from src.models.validator import validate
from src.utils.utils_trainer import get_batch_caps, get_hypothesis, adjust_learning_rate
from torchvision import transforms
from sklearn.preprocessing import OneHotEncoder


# Training Model

In [ ]:
use_exisiting_model  = False

if use_exisiting_model:
    current_time = "2022-04-02-02-25-40"
    encoder_file = 'encoder-49.pkl' 
    decoder_file = 'decoder-49.pkl'

In [ ]:
from datetime import datetime

if use_exisiting_model:
    log_train = f'./logs/{current_time}/training_log.txt'       # name of files with saved training loss and perplexity
    log_val = f'./logs/{current_time}/validation_log.txt'
    bleu = f'./logs/{current_time}/bleu.txt'
    vocab_file = f'./assets/{current_time}/vocab.pkl' 

    with open(bleu, 'r') as f: 
        records = f.readlines()
    bleu_scores = []
    for record in records:
        bleus = []
        for blue in record.split(',')[-4:]:
            bleus.append(float(blue.split(":")[-1].strip()))
        bleu_scores.append(bleus)

    with open(f'../models_new/{current_time}/onehot_cat_encoder', "rb") as f:
        onehot_cat_encoder = pickle.load(f)
    
    with open(f'../models_new/{current_time}/onehot_seq_encoder', "rb") as f:
        onehot_seq_encoder = pickle.load(f)

    # Open the training log file.
    file_train = open(log_train, 'a')
    file_val = open(log_val, 'a')
    bleu_score_file = open(bleu, 'a')
    start_epoch = int(encoder_file.split('.')[0].split('-')[-1])+1
    print(start_epoch)
    
else:
    now = datetime.now()
    current_time = now.strftime("%Y-%m-%d-%H-%M-%S")
    if not os.path.exists(f'../models_new/{current_time}'):
        os.makedirs(f'../models_new/{current_time}')
    if not os.path.exists(f'./logs/{current_time}'):
        os.makedirs(f'./logs/{current_time}')
    if not os.path.exists(f'./assets/{current_time}'):
        os.makedirs(f'./assets/{current_time}')
    
    vocab_file = f'./assets/{current_time}/vocab.pkl' 
    log_train = f'./logs/{current_time}/training_log.txt'       # name of files with saved training loss and perplexity
    log_val = f'./logs/{current_time}/validation_log.txt'
    bleu = f'./logs/{current_time}/bleu.txt'
    
    # Open the training log file.
    file_train = open(log_train, 'w')
    file_val = open(log_val, 'w')
    bleu_score_file = open(bleu, 'w')

    start_epoch= 0 
    bleu_scores = []

    image_dict = product_info["imageURLHighRes"].to_dict()
    caption_dict = product_info["description"].to_dict()
    category_dict = product_info["perCategory"].to_dict()
    sentence_id_dict = product_info["sentence_index"].to_dict()

    onehot_cat_encoder = OneHotEncoder(handle_unknown='ignore').fit(np.array([*category_dict.values()], dtype=object).reshape(-1, 1))
    with open(f'../models_new/{current_time}/onehot_cat_encoder', "wb") as f: 
        pickle.dump(onehot_cat_encoder, f)
    
    onehot_seq_encoder = OneHotEncoder(handle_unknown='ignore').fit(np.array([*sentence_id_dict.values()], dtype=object).reshape(-1, 1))
    with open(f'../models_new/{current_time}/onehot_seq_encoder', "wb") as f: 
        pickle.dump(onehot_seq_encoder, f)

In [ ]:
batch_size = 10          # batch size, change to 64
vocab_threshold = 5        # minimum word count threshold
vocab_from_file = False    # if True, load existing vocab file
embed_size = 125           # dimensionality of image and word embeddings
hidden_size = 512          # number of features in hidden state of the RNN decoder
num_features = 2048        # number of feature maps, produced by Encoder
num_epochs = 200               # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss

In [ ]:
train_df, val_test_df = train_test_split(product_info, test_size=0.4)
train_df.reset_index(drop=True,inplace=True)
valid_df, test_df = train_test_split(val_test_df, test_size=0.5)
valid_df.reset_index(drop=True,inplace=True)
test_df.reset_index(drop=True,inplace=True)

# transform_train = transforms.Compose([ 
#     transforms.Resize(500),                          # smaller edge of image resized to 256
#     transforms.Resize((320,320)),                      # get 224x224 crop from random location
#     transforms.ToTensor(),                           # convert the PIL Image to a tensor
#     transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
#                          (0.229, 0.224, 0.225))])
transform_train = transforms.Compose([ 
    transforms.Resize(500),                          # smaller edge of image resized to 256
    transforms.Resize((320,320)),                      # get 224x224 crop from random location
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
])


# Build data loader.
data_loader = get_loader(train_df,
                        transform=transform_train,
                        onehot_cat_encoder=onehot_cat_encoder,
                        mode='train',
                        onehot_seq_encoder=onehot_seq_encoder,
                        image_type="imageURLHighRes",
                        caption_type="description",
                        vocab_file = vocab_file,
                        batch_size=batch_size,
                        vocab_threshold=vocab_threshold,
                        vocab_from_file=use_exisiting_model)


# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)
print(vocab_size)

print('The shape of first image:', data_loader.dataset[0][0].shape)
print('Total number of tokens in vocabulary:', len(data_loader.dataset.vocab))

# Randomly sample a caption length, and sample indices with that length.
indices = data_loader.dataset.get_indices()
print('sampled indices:', indices)

# Create and assign a batch sampler to retrieve a batch with the sampled indices.
new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
data_loader.batch_sampler.sampler = new_sampler
    
# Obtain the batch.
images, onehot_cat, onehot_enc_seq, captions = next(iter(data_loader))
    
print('images.shape:', images.shape)
print('onehot_cat.shape:', onehot_cat.shape)
print('captions.shape:', captions.shape)

plt.axis('off')
plt.imshow(images[0].detach().T)
plt.show()

# Setup the transforms
transform_test = transforms.Compose([ 
    transforms.Resize(500),                          # smaller edge of image resized to 256
    transforms.Resize((320,320)),                      # get 224x224 crop from random location
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
])

# Create the data loader.
valid_data_loader = get_loader(valid_df,
                        transform=transform_train,
                        onehot_cat_encoder=onehot_cat_encoder,
                        mode='valid',
                        onehot_seq_encoder=onehot_seq_encoder,
                        image_type="imageURLHighRes",
                        caption_type="description",
                        vocab_file = vocab_file,
                        batch_size=batch_size)


total_step_valid = math.ceil(len(valid_data_loader.dataset.caption_lengths) / valid_data_loader.batch_sampler.batch_size)
total_step_valid

In [ ]:
encoder = EncoderCNN()
decoder = DecoderRNN(num_features = num_features, 
                    embedding_dim = embed_size, 
                    category_dim = len(set(category_dict.values())) + len(set(sentence_id_dict.values())) ,
                    hidden_dim = hidden_size, 
                    vocab_size = vocab_size,
                    cat_attention = True)


if use_exisiting_model:
    encoder.load_state_dict(torch.load(os.path.join(f'../models_new/{current_time}', encoder_file), map_location='cpu'))
    decoder.load_state_dict(torch.load(os.path.join(f'../models_new/{current_time}', decoder_file), map_location='cpu'))



# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

In [ ]:
print(device)
# Define the loss function. 
criterion = nn.CrossEntropyLoss()
criterion.to(device)
#params = list(decoder.parameters()) + list(encoder.parameters()) 
params = list(decoder.parameters())

# TODO #4: Define the optimizer.
grad_clip = 5.  # clip gradients at an absolute value of
alpha_c = 1.  # regularization parameter for 'doubly stochastic attention', as in the paper

encoder_lr = 1e-3  # learning rate for encoder if fine-tuning
encoder_optimizer = torch.optim.Adam(params=filter(lambda p: p.requires_grad, encoder.parameters()),lr=encoder_lr)
encoder_lr_scheduler = torch.optim.lr_scheduler.CyclicLR(encoder_optimizer, 
                                                         base_lr=encoder_lr/10, max_lr=encoder_lr*10, 
                                                         step_size_up = 10 , step_size_down=20, cycle_momentum=False)


decoder_lr = 1e-3  # learning rate for decoder
decoder_optimizer = torch.optim.Adam(params=filter(lambda p: p.requires_grad, decoder.parameters()),lr=decoder_lr)
decoder_lr_scheduler = torch.optim.lr_scheduler.CyclicLR(decoder_optimizer, 
                                                         base_lr=decoder_lr/10, max_lr=decoder_lr*10, 
                                                         step_size_up = 10 , step_size_down=20, cycle_momentum=False)

fine_tune_encoder = False  # fine-tune encoder?
checkpoint = None  # path to checkpoint, None if none

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)


In [ ]:

# store BLEU scores in list 

total_step_valid = math.ceil(len(valid_data_loader.dataset.caption_lengths) / valid_data_loader.batch_sampler.batch_size)
epochs_since_improvement = 0
best_bleu = 0.

for epoch in range(start_epoch, num_epochs+1):
    # Decay learning rate if there is no improvement for 8 consecutive epochs, and terminate training after 20
    if epochs_since_improvement == 50:
        break
    if epochs_since_improvement > 0 and epochs_since_improvement % 20 == 0:
        adjust_learning_rate(decoder_optimizer, 0.8)
        if fine_tune_encoder:
            adjust_learning_rate(encoder_optimizer, 0.8)

    train(epoch, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, total_step, num_epochs =num_epochs,
          data_loader = data_loader,
          write_file = file_train, 
          vocab_size = vocab_size,
          batch_size = batch_size,
          device = device,
          print_every = print_every,
          save_every = 1)
    
    bleus= validate(epoch, encoder, decoder, 
                    encoder_optimizer, decoder_optimizer, criterion, 
                    vocab_size = vocab_size,
                    batch_size = batch_size,
                    device = device,
                    print_every = print_every,
                    total_step = total_step_valid, 
                    num_epochs = num_epochs, 
                    data_loader = valid_data_loader, 
                    write_file=file_val, 
                    bleu_score_file=bleu_score_file)
    
    bleu_scores.append(bleus)
    # encoder_lr_scheduler.step()
    # decoder_lr_scheduler.step()
    
    # Check if there was an improvement
    is_best = sum(bleus)/4 > best_bleu
    best_bleu = max(sum(bleus)/4, best_bleu)
    if not is_best:
        epochs_since_improvement += 1
        print("\nEpochs since last improvement: %d\n" % (epochs_since_improvement,))
    else:
        torch.save(decoder.state_dict(), os.path.join(f'../models_new/{current_time}', 'decoder-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join(f'../models_new/{current_time}', 'encoder-%d.pkl' % epoch))
        epochs_since_improvement = 0
    
file_train.close()
file_val.close()
bleu_score_file.close()

# Generate Predictions

In [ ]:
# Setup the transforms
# transform_test = transforms.Compose([ 
#     transforms.Resize((320,320)),                          # smaller edge of image resized to 256
#     transforms.ToTensor(),                           # convert the PIL Image to a tensor
#     transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
#                          (0.229, 0.224, 0.225))])

transform_test = transforms.Compose([ 
    transforms.Resize(500),                          # smaller edge of image resized to 256
    transforms.Resize((320,320)),                      # get 224x224 crop from random location
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
])

In [ ]:
test_data_loader = get_loader(test_df,
                              image_type="imageURLHighRes",
                              caption_type="description",
                              transform=transform_test,
                              onehot_cat_encoder=onehot_cat_encoder,
                              onehot_seq_encoder=onehot_seq_encoder,
                              batch_size=1,
                              mode='test')

In [ ]:
encoder_file = 'encoder-0.pkl' 
decoder_file = 'decoder-0.pkl'

embed_size = 125           # dimensionality of image and word embeddings
hidden_size = 512          # number of features in hidden state of the RNN decoder
num_features = 2048        # number of feature maps, produced by Encoder

# The size of the vocabulary.
vocab_size = 3633#len(test_data_loader.dataset.vocab)

# Initialize the encoder and decoder, and set each to inference mode.
encoder = EncoderCNN()
encoder.eval()
decoder = DecoderRNN(num_features = num_features, 
                     embedding_dim = embed_size,
                     category_dim = len(set(category_dict.values())) + len(set(sentence_id_dict.values())),
                     hidden_dim = hidden_size, 
                     vocab_size = vocab_size)
decoder.eval()

current_time = "2022-03-31-16-43-01"
# Load the trained weights.
encoder.load_state_dict(torch.load(os.path.join(f'../models_new/{current_time}', encoder_file), map_location='cpu'))
decoder.load_state_dict(torch.load(os.path.join(f'../models_new/{current_time}', decoder_file), map_location='cpu'))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move models to GPU if CUDA is available.
encoder.to(device)
decoder.to(device)

In [ ]:
def clean_sentence(output, data_loader):
    vocab = data_loader.dataset.vocab.idx2word
    words = [vocab.get(idx) for idx in output]
    words = [word for word in words if word not in (',', '.', '<end>')]
    sentence = " ".join(words)
    
    return sentence

def get_prediction(data_loader):
    orig_image, image, cat_onehot, seq_onehot, caption= next(iter(data_loader))
    plt.imshow(orig_image.squeeze())
    print(cat_onehot.size())
    plt.title(caption)
    plt.show()
    image = image.to(device)
    onehot = torch.cat((cat_onehot.view(1,-1), 
                            seq_onehot.view(1,-1)), 1).type('torch.FloatTensor').view(1,-1).to(device)
    
    features = encoder(image)
    output, atten_weights = decoder.greedy_search(features, onehot)    
    sentence = clean_sentence(output,data_loader)
    print(sentence)

In [ ]:
get_prediction(test_data_loader)